In [ ]:
import pandas as pd
import joblib
import warnings
from pathlib import Path
from sklearn.model_selection import train_test_split
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights
from responsibleai.feature_metadata import FeatureMetadata

warnings.filterwarnings('ignore')

current_dir = Path().resolve()

if "src" in [p.name for p in current_dir.parents]:
    src_dir = [p for p in current_dir.parents if p.name == "src"][0]
    PROJECT_ROOT = src_dir.parent
else:
    PROJECT_ROOT = current_dir

ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
REPORTS_DIR = PROJECT_ROOT / 'reports'

SAMPLE_SIZE = 5000
RANDOM_STATE = 42

numerical_cols = ['Age_Years', 'Systolic_BP', 'Diastolic_BP', 'BMI', 'Pulse_Pressure']
categorical_cols = ['Sex', 'Cholesterol_Level', 'Glucose_Level', 'Smoking_Status',
                    'Alcohol_Intake', 'Physical_Activity', 'BP_level']

scaler_path = ARTIFACTS_DIR / 'robust_scaler_numeric.pkl'
robust_scaler = joblib.load(scaler_path)

_, X_train_rai, _, y_train_rai = train_test_split(
    X_train_full, y_train,
    test_size=SAMPLE_SIZE,
    stratify=y_train,
    random_state=RANDOM_STATE
)

_, X_test_rai, _, y_test_rai = train_test_split(
    X_test_full, y_test,
    test_size=SAMPLE_SIZE,
    stratify=y_test,
    random_state=RANDOM_STATE
)

def inverse_transform_features(scaled_df, numerical_cols, scaler):
    original_df = scaled_df.copy()
    original_df[numerical_cols] = scaler.inverse_transform(original_df[numerical_cols])
    return original_df

X_train_rai_original = inverse_transform_features(X_train_rai, numerical_cols, robust_scaler)
X_test_rai_original = inverse_transform_features(X_test_rai, numerical_cols, robust_scaler)

X_train_rai_original['target'] = y_train_rai.values
X_test_rai_original['target'] = y_test_rai.values

feature_metadata = FeatureMetadata()
for feature in categorical_cols:
    if feature in X_train_rai_original.columns:
        feature_metadata.add_feature(feature, 'categorical')

rai_insights = RAIInsights(
    model=calibrated_model,
    train=X_train_rai_original,
    test=X_test_rai_original,
    target_column='target',
    task_type='classification',
    feature_metadata=feature_metadata
)

sensitive_features = X_test_rai_original[['Sex']]

rai_insights.error_analysis.add()
rai_insights.model_overview.add()
rai_insights.explainer.add()
rai_insights.fairness.add(sensitive_features=sensitive_features)

rai_insights.compute()

ResponsibleAIDashboard(rai_insights)

p:\projects\heart_disease_prediction\rai_env\lib\site-packages\llvmlite\binding\ffi.py:175: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


SystemError: initialization of _internal failed without raising an exception